In [4]:
pred_values = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [5]:
import os
import numpy as np
from scipy.io import wavfile
import pandas as pd

import operator
from pydub import AudioSegment
from audio.processor import WavProcessor, format_predictions
from collections import Counter

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
df = pd.read_csv('labels.csv')
df = df.set_index('index')

sig = AudioSegment.from_file('workshop1.wav', format="wav")

window_size = 6 * 1000
overlap = 3 * 1000
fragment_length = 900 * 1000

current_length = 0

window_start = 0
window_end = window_start + window_size

codes = []

def get_labels(idx): 
    return [0.0 if val != 1 else 1.0 for val in df.loc[idx].values]

while current_length < fragment_length:
    print(window_start,window_end)
    
    segment = sig[window_start:window_end]
    samples = np.array(segment.get_array_of_samples())
    new_sig = samples.astype(np.float32)

    with WavProcessor() as proc:
        predictions = proc.get_predictions(16000, new_sig)

        for p in predictions:
            codes.append(p)
            pred_values = [x + y for x, y in zip(pred_values, get_labels(p[0]))]
            print(str(window_start / (60*1000)), ":", str(window_end / (60*1000)), " - ", df.loc[p[0]].values[1], p[1])
    
    current_length = window_start
    window_start += (window_size - overlap)
    window_end = (window_start + window_size)

0 6000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
3000 9000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
6000 12000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
0.1 : 0.2  -  Whir 0.40222224593162537
0.1 : 0.2  -  White noise 0.1019059345126152
9000 15000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
0.15 : 0.25  -  Vehicle 0.30564165115356445
12000 18000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
15000 21000
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt


In [22]:
val_list = {}

total_items = 0

for c in codes:
    total_items += 1
    
    if c[0] not in val_list:
        val_list[c[0]] = []
        
    val_list[c[0]].append(c[1])
    
items_dict = {}

for key, val in val_list.items():
    harmonic = 2 / (1 / (len(val)/total_items) + 1 / np.mean(val))
    items_dict[key] = harmonic

_sorted = sorted(items_dict.items(), key=lambda x: x[1])

for key, val in _sorted:
    print(df.loc[key]['display_name'], val)

Gong 0.013938799479990079
Toot 0.014156263427075893
Car 0.01420993396004451
Truck 0.014242520215207378
Aircraft 0.014403116513302446
Chime 0.01445237830570302
Propeller, airscrew 0.014472274368905707
Chirp tone 0.02788015487708804
Medium engine (mid frequency) 0.028502069644511616
Television 0.040436301370872015
Outside, urban or manmade 0.05235741221025435
Car passing by 0.05338618981183786
Microwave oven 0.0641442832865583
Music 0.06828437603117801
Burst, pop 0.08298582515430292
Whir 0.10350811534987718
White noise 0.1272209059975423
Explosion 0.138944444802364
Vehicle 0.21840101594929012
Static 0.4636432675333752


In [23]:
val_list = {}

total_items = 0

for c in codes:
    total_items += 1
    
    if c[0] not in val_list:
        val_list[c[0]] = []
        
    val_list[c[0]].append(c[1])

items_dict = {}

for key, val in val_list.items():
    mean = (len(val)/total_items) * np.mean(val)
    items_dict[key] = mean

_sorted = sorted(items_dict.items(), key=lambda x: x[1])

for key, val in _sorted:
    print(df.loc[key]['display_name'], val)

Gong 0.0007868396924502814
Toot 0.001025063444429369
Car 0.0011061391056473575
Truck 0.0011615825455580184
Aircraft 0.0015358999371528625
Chime 0.001701662122313656
Propeller, airscrew 0.0017788294535964282
Chirp tone 0.003151729925354915
Television 0.004670171372926057
Medium engine (mid frequency) 0.00470781037166937
Outside, urban or manmade 0.006352594594902067
Car passing by 0.0075326040387153625
Microwave oven 0.008519617256833544
Burst, pop 0.01053702175172407
Music 0.014989667196771991
Whir 0.015150039554087084
White noise 0.023196553561224867
Explosion 0.03710389559838309
Vehicle 0.06736444715243667
Static 0.2513848706635077


In [101]:
pred_values

[0.0,
 0.0,
 2.0,
 2.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 2.0,
 2.0,
 2.0,
 0.0,
 4.0,
 2.0,
 37.0,
 29.0,
 33.0]

In [16]:
df.loc[137]

mid                    /m/04rlf
display_name              Music
Living room                   1
Work place(kantoor)           1
Classroom                   NaN
Work Shop                   NaN
Kitchen                     NaN
Bathroom                    NaN
Bedroom                     NaN
Trainstation                NaN
Airfield                    NaN
On water                    NaN
Public place                NaN
Roadside                    NaN
Concert                       1
Farm                        NaN
Nature                      NaN
Park                        NaN
Name: 137, dtype: object

In [103]:
arr_index = np.argwhere(pred_values == np.amax(pred_values))
arr_index = arr_index.flatten().tolist()

labels = ['Living room', 'Work place(kantoor)', 'Classroom', 'Work Shop', 'Kitchen', 'Bathroom', 'Bedroom', 'Trainstation', 'Airfield', 'On water', 'Public place', 'Roadside', 'Concert', 'Farm', 'Nature', 'Park']

for x in arr_index:
    print(labels[int(x)])

print(pred_values)

Park
[0.0, 0.0, 2.0, 2.0, 0.0, 1.0, 1.0, 1.0, 0.0, 2.0, 2.0, 2.0, 0.0, 4.0, 2.0, 37.0, 29.0, 33.0]
